# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_mixed")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!
Cloning into 'gpt-neo-125M'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 44 (delta 20), reused 0 (delta 0)
Unpacking objects: 100% (44/44), 543.14 KiB | 1.21 MiB/s, done.
Using framework PyTorch: 1.10.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTor

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cuda:0


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(2):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon. I can fly, but I can't swim. I've got a four-headed dragon, so I'm not sure if I can do that."<p><d>"I can't swim, I'm afraid."<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon. I can see that."<|endoftext|>
PyTorch: In my dreams, I'm a dragon. I've dreamed of dragons before, and I'm not sure what I'm going to find out."<d><scn>o2<msg>I'm not sure if there is anything in particular that I could do to help, but I'm not taking any samples. I'm not taking any samples. I'm not sure what the world record holder for "most human" would be, but I'm not taking any samples. I'm not sure if there is anything in particular that could be useful to add to the archives or what they would like to see me do."<|endoftext|>
----------------------------------------------------------------------------------------------------
ONNX: In my dreams, I'm a dragon. I can fly, but I can't swim. I've got a four-headed dragon, so I'm not sure if I can do that."<d><scn>beach

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [6]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hello, [player_name]."', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm fine. I just have a few questions about the murder of your colleague."<d><scn>park2<msg>Ry "I'm afraid so."<d><scn>park2<msg>Ry "I'm afraid so, but I have to admit, I was kinda annoyed about the whole thing. I guess it showed."<

[ONNX] Reply: park2<msg>Ry "I'm fine. I just have a few questions about the murder of your colleague."<d><scn>park2<msg>Ry "I'm afraid so."<d><scn>park2<msg>Ry "I'm afraid so, but I have to admit, I was kinda annoyed about the whole thing. I guess it showed."<d><scn>park2<msg>Ry "I'm not sure about that."<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hello, [player_

[ONNX Quantized] Reply: park2<msg>Ry "I'm fine. I'm just a little busy."<p><msg>c "What about you, Ipsum?"<d><scn>park2<msg>Ip "I was just kidding."<p><msg>c "I was just kidding."<d><scn>park2<msg>Ry "I was just kidding. I was just kidding."<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he's f

# Sampling test

This is gonna be interesting!

In [7]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm fine. I'm just sad to leave you alone with me."<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hello, [player_name]."<p><msg>c "What are you talking about?"<d><scn>park2<msg>Ry "Do you consider

[ONNX] Reply: park2<msg>Ry "I'm good, really. Thanks for your help."<d><scn>park2<msg>Ry "Likewise!"<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ry "I'm not. What are you talking about?"<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he's funny."<|endoftext|>

[ONNX] Reply: park2<msg>Ad "I think he's cute."<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ad "Not really. I think he's good look."<|endoftext|>

----------
Prompt: Oh my god, Adine. What is this?
[Pytorch] Reply: o2<msg>Ad "It's a long story."<p><msg>c "Go to Emera"<d><scn>emeraroom<msg>Em "Oh, [player_name], I didn't expect you to arrive so early."<|endoftext|>

[ONNX] Reply: o2<msg>Ad "It's some kind of joke I saw you do."<p

# RP test
Testing out the injected roleplay actions

In [8]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Oh, [player_name], I didn't expect you to arrive so early."<|endoftext|>
[ONNX] Visit Lorem -> loremapt<msg>Lo "Oh. This is going pretty long. I'll have more than enough time to get this done when I'm done."<p><msg>c "I'm not sure if it's worth it to ask you to take a seat."<d><scn>loremapt<msg>Lo "Oh, right. I see."<d><scn>loremapt<msg>Lo "I'm sorry, but I can't sell my body for an I am."<p><msg>c "Hey, Lorem. What's the problem?"<d><sc
[ONNX Quantized] Visit Lorem -> loremapt<msg>Lo "I thought it would show up as soon as it was supposed."<p><msg>c "Then it should show you some proof."<d><scn>loremapt<msg>Lo "No. If you can't prove it, you'll want a lawyer."<p><msg>c "I'm not sure I have anything to offer."<d><scn>loremapt<msg>Lo "I'll have a few things for you to do."<p><msg>c "hello kreak"<d><scn>loremapt
----------
[Pytorch] Meet with Lorem -> loremapt<msg>Lo "Hey [player_name]!"<|endoftext|>
[ONNX] Meet with Lorem -> loremapt<msg>Lo "Hey [